# Support Vector Machines

We’ve attached a dataset, $\texttt{MNISTdata.mat}$, containing a sample of the famout MNISTbenchmark (http://yann.lecun/com/exdb/mnist).   Your  report  must  provide  summaries  of  each  method’s  performance  andsome  additional  details  of  your  implementation.   Compare  the  relative  strengths  andweaknesses of the methods based on both the experimental results and your understandingof the algorithms. 

You can load the data with $\texttt{scipy.io.loadmat}$, which will return a Python dictionarycontaining the test and train data and labels.

The purpose of this assignment is for you to implement the SVM. You are not allowed toimport an SVM from, for instance, $\texttt{scikit-learn}$.  You may, however, use a library (suchas $\texttt{scipy.optimize.minimize}$ or $\texttt{cvxopt.solvers.qp}$) for the optimization process.

Please refer to the instructions in the PDF document coming with the writen problems. 

### (a) SVM for binary classifications

Develop code for training an SVM for binary classification with nonlinear kernels

In [98]:
import scipy.io as sci
import numpy as np
from cvxopt import solvers
from cvxopt import matrix as cvxopt_matrix
from math import e
f = sci.loadmat("MNIST_data.mat")
train_sample = f['train_samples']
train_label = f['train_samples_labels']
test_sample = f['test_samples']
test_label = f['test_samples_labels']
train_data = np.hstack([train_sample,train_label])


def svm(data,num1, num2):
    Y = data[:,-1]
    for i in range(Y.shape[0]):
        if Y[i] == num1:
            Y[i] = 1
        else:
            Y[i] = -1

    X = data[:,:-1]
    n,m = X.shape
    K = np.matmul(X,X.transpose())
    K = K + 2
    K = np.square(K)
    
    T = np.diag(Y)
    P = cvxopt_matrix(np.matmul(np.matmul(T,K),T))
    q = cvxopt_matrix(-np.ones((n)))
    G = cvxopt_matrix(-np.eye(n))
    h = cvxopt_matrix(np.zeros(n))
    b = cvxopt_matrix(np.zeros(1))
    A = cvxopt_matrix(np.asmatrix(Y))
    solvers.options['show_progress'] = False
    alpha = np.array(solvers.qp(P,q,G,h,A,b)['x'])
    w = sum([alpha[i]*Y[i]*X[i,:] for i in range(n)])
    b = 1/n*(sum([Y[i]-np.dot(w,X[i,:]) for i in range(n)]))
    return w,b


### (b) Predict for new data

Develop code for predict the $\{-1, +1\}$ for new data. To use the predictive model (7.13) you need to determine $b$, which can be done with (7.37).

In [95]:
def predict(train_sample, test_sample,num1,num2):
    w,b = svm(train_sample,num1,num2)
    print(b)
    result = []
    for i in range(test_sample.shape[0]):
        result.append(np.sign(np.dot(w,test_sample[i,:])+b))
    return result


### (c) Multiclass classification

Useing your implementation, compare multiclass classification performance of two different voting schemes: 
 i. one versus rest;
 ii. one versus one

In [79]:
final_result = [] 
for i in range(10):
    f = sci.loadmat("MNIST_data.mat")
    train_sample = f['train_samples']
    train_label = f['train_samples_labels']
    test_sample = f['test_samples']
    test_label = f['test_samples_labels']
    train_data = np.hstack([train_sample,train_label])
    w,b = svm(train_data,i,num2 = None)
    final_result += [np.append(w,b)]


In [90]:
result = []
for i in test_sample:
    lst = []
    count = 0
    for j in final_result:
        lst += [[np.matmul(i.transpose(),j[:-1])+j[-1],count]]
        
        count += 1
    result += [max(lst)[1]]


In [91]:
count = 0
for i in range(len(result)):
    if result[i] == test_label[i]:
        count += 1 
print(count/len(result))
confusion_matrix =np.diag([0]*10)
for i in range(len(result)):
    confusion_matrix[int(result[i]),int(test_label[i,0])] += 1
print(confusion_matrix)


0.772
[[ 84   0   6   2   1  16   4   1   7   1]
 [  0 118   7   2   0   3   1   1   2   1]
 [  0   0  78   1   0   2   2   0   1   0]
 [  0   0   7  86   0   6   0   2   3   4]
 [  0   0   0   3  98   9   0   1   8  18]
 [  0   0   0   6   0  44   2   0   1   0]
 [  1   1   3   6   2   1  76   0   2   0]
 [  0   0   5   7   3   6   2  86   5  19]
 [  0   3   5   0   1   2   0   0  54   1]
 [  1   0   2   2   3   3   0   8   3  48]]


In [99]:
#one vs one

model_set = []
for i in range(10):
    for k in range(0,10):
        if k != i:
            pos_sample = []
            pos_label = []
            for j in range(len(train_label)):
                if train_label[j] == i or train_label[j] == k:
                    pos_sample.append(train_sample[j])
                    pos_label.append(train_label[j])
            model_set += [[svm(np.hstack([np.array(pos_sample),np.array(pos_label)]),i,k),i,k]]
    
predict = []
for i in range(test_label.shape[0]):
    lst = {}
    for j in range(90):
        w,b = model_set[j][0]
        if np.dot(w,test_sample[i]) + b > 0:
            if model_set[j][1] not in lst:
                lst[model_set[j][1]] = 1
            else:
                lst[model_set[j][1]] += 1
    
            
    predict += [max(lst, key=lst.get)]


In [101]:
count = 0
for i in range(len(predict)):
    if predict[i] == test_label[i]:
        count += 1 
print(count/len(predict))
confusion_matrix =np.diag([0]*10)
for i in range(len(predict)):
    confusion_matrix[int(predict[i]),int(test_label[i,0])] += 1
print(confusion_matrix)



0.9
[[ 84   0   0   0   0   1   3   0   1   0]
 [  0 122   5   3   0   2   0   1   3   2]
 [  1   0  96   1   0   0   1   3   0   0]
 [  0   0   0  98   0   5   0   1   4   0]
 [  0   0   1   0 101   2   0   2   4   3]
 [  0   0   0   5   0  77   1   0   0   0]
 [  1   0   1   1   2   1  82   0   0   0]
 [  0   0   3   4   0   0   0  89   3   2]
 [  0   0   4   2   0   4   0   0  67   1]
 [  0   0   3   1   5   0   0   3   4  84]]


In my implement, one-vs-one is strictly much faster than one-vs-rest method and even more accurate. The reason that one-vs-one is faster than one-vs-rest maybe because one-vs-rest's quadratic problem cost much more time as size of data is larger.


### (d) Hyperparameter

The parameter $C >0$ controls the tradeoff between the size of the margin and theslack variable penalty.  It is analogous to the inverse of a regularization coefficient.Include in your report a brief discussion of how you found an appropriate value.

As C is closer to inf, the more overfit hyperplane is. the opposite otherwise. In order to find an appropriate value for C, we can form a integer loop from 0 to 1000 and inf and generate optimal solution each time and check accuracy rate of how the optimal solution alpha perform in the binary classification and finally choose one integer that perform the best.

### (e) Confusion matrix

In addition to calculating percent accuracy, generate multiclass confusion matrices (https://en.wikipedia.org/wiki/Confusionmatrix) as part of your analysis.